### Import libraries and the review file

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/teohangxanh/Practice-Data-Science/master/FarLandMD/zocdoc%20reviews.csv', encoding = "ISO-8859-1")
pd.set_option('max_colwidth', 200)
print(df.shape)

(71, 2)


### Clean the dataset

In [2]:
df.head()

,Doctor,Reviews
0,"Dr. Jon Biorkman, MD",['This review is an overdue huge thank you for an extraordinary medical doctor whose care and professionalism helped me even in his absence: I was Dr. Biorkman\'s patient while in Irvine until 20...
1,NaN,NaN
2,"Richard McConkie, FNP-C","[""I was so pleased with the kindness and care of the staff at west valley. This was my first visit and it felt like I'd been going there forever. Dr Richard was very empathic and understanding. Li..."
3,NaN,NaN
4,"Dr. Crystal Song, NMD","[""I go monthly to Dr Song to strengthen my abdomen which has a hernia caused by a botched surgery 2 years ago. I am extremely satisfied by the high expertise and genuine care that Dr Song gives m..."


In [3]:
# Remove rows that have missing data in all columns: Doctor and Reviews
df.dropna(how='all', inplace=True)

In [4]:
df.head()

,Doctor,Reviews
0,"Dr. Jon Biorkman, MD",['This review is an overdue huge thank you for an extraordinary medical doctor whose care and professionalism helped me even in his absence: I was Dr. Biorkman\'s patient while in Irvine until 20...
2,"Richard McConkie, FNP-C","[""I was so pleased with the kindness and care of the staff at west valley. This was my first visit and it felt like I'd been going there forever. Dr Richard was very empathic and understanding. Li..."
4,"Dr. Crystal Song, NMD","[""I go monthly to Dr Song to strengthen my abdomen which has a hernia caused by a botched surgery 2 years ago. I am extremely satisfied by the high expertise and genuine care that Dr Song gives m..."
6,"Dr. Christopher Ciccone, MD",['Excellent service and advice as well as a perscrition for spider bite infection\n\nDr. Ciccone has been my doc for 25 years and is now the family doc for my children and husband too! His offic...
8,"Dr. Martin Maag, MD","['Very positive and encouraging, time well spent.\n\nFriendly, professional, and knowledgeable.\n\nits not often that the Doctor is the one to greet you and sit you down in the consultation room, ..."


In [5]:
pattern = '|'.join(['\[', '\]', '\\\\\\', '\'', "\'", '\\n', '\\\\', '  '])
df.Reviews = df.Reviews.str.replace(pattern, '')

In [6]:
df.iloc[1, 1]

'"I was so pleased with the kindness and care of the staff at west valley. This was my first visit and it felt like Id been going there forever. Dr Richard was very empathic and understanding. Listened to my issues and was a wonderful breath of fresh air in this crazy world we find ourselves in. Thank you again.", Excellent! Staff was helpful and considerate. Richard was amazingly kind and patient with my handicapped daughter! Would definitely recommend their office!Office was super clean and staff was very friendly! The provider was great! I would recommend to all my friends and family.My son and I are new patients of Dr. McConkie, at our hometown West Valley Med. Clinic in Middleton. From the moment you walk in the door they made us both feel like we were right at home, and Dr. McConkie was very thorough with addressing all of our concerns, nThank you Dr. McConkie!Always pleasant.Richard does a great job and as long as he is in the Middleton Clinic, I will travel there for my appoint

In [7]:
import spacy
from spacy.lang.en import English

nlp = spacy.load('en_core_web_lg')
doc = nlp(df.iloc[1, 1])

In [8]:
# Create a list of word tokens
filtered_words = []
for token in doc:
    # Remove stop words
    if token.is_stop == False and token.is_punct == False:
        filtered_words.append(token.text)
print(filtered_words)

['pleased', 'kindness', 'care', 'staff', 'west', 'valley', 'visit', 'felt', 'like', 'd', 'going', 'forever', 'Dr', 'Richard', 'empathic', 'understanding', 'Listened', 'issues', 'wonderful', 'breath', 'fresh', 'air', 'crazy', 'world', 'find', 'Thank', 'Excellent', 'Staff', 'helpful', 'considerate', 'Richard', 'amazingly', 'kind', 'patient', 'handicapped', 'daughter', 'definitely', 'recommend', 'office!Office', 'super', 'clean', 'staff', 'friendly', 'provider', 'great', 'recommend', 'friends', 'family', 'son', 'new', 'patients', 'Dr.', 'McConkie', 'hometown', 'West', 'Valley', 'Med', 'Clinic', 'Middleton', 'moment', 'walk', 'door', 'feel', 'like', 'right', 'home', 'Dr.', 'McConkie', 'thorough', 'addressing', 'concerns', 'nThank', 'Dr.', 'McConkie!Always', 'pleasant', 'Richard', 'great', 'job', 'long', 'Middleton', 'Clinic', 'travel', 'appointments', 'chronic', 'cough', 'months-', 'saw', 'providers-', 'provider', 'attentive', 'knowledgeable', 'ordered', 'treatment', 'instead', 'usual-"let

### Can we create a filtered adjectives which have positive or negative meanings of the doctor, not irrelevant ones?

In [9]:
# Create a list of sentence tokens

# Create the pipeline 'sentencizer' component
sbd = nlp.create_pipe('sentencizer')

# Add the component to the pipeline
nlp.add_pipe(sbd)

# create list of sentence tokens
sents_list = []
for sent in doc.sents:
    sents_list.append(sent.text)
print(sents_list)

['"I was so pleased with the kindness and care of the staff at west valley.', 'This was my first visit and it felt like', 'Id been going there forever.', 'Dr Richard was very empathic and understanding.', 'Listened to my issues and was a wonderful breath of fresh air in this crazy world we find ourselves in.', 'Thank you again.", Excellent!', 'Staff was helpful and considerate.', 'Richard was amazingly kind and patient with my handicapped daughter!', 'Would definitely recommend their office!Office was super clean and staff was very friendly!', 'The provider was great!', 'I would recommend to all my friends and family.', 'My son and I are new patients of Dr. McConkie, at our hometown West Valley Med. Clinic in Middleton.', 'From the moment you walk in the door', 'they made us both feel like we were right at home, and Dr. McConkie was very thorough with addressing all of our concerns, nThank you Dr. McConkie!Always pleasant.', 'Richard does a great job and as long as he is in the Middlet

In [10]:
# for chunk in doc.noun_chunks:
#     print('%40s' % chunk.text, '%10s' % chunk.root.text, '%10s' % chunk.root.dep_, '%10s' % chunk.root.head.text, sep='\t')

In [11]:
from spacy.matcher import Matcher
doc = nlp(df.iloc[1, 1])
matcher = Matcher(nlp.vocab)
p1 = [{'ORTH': 'Dr.', 'OP': '*'},
           {'ENT_TYPE': 'PERSON'},
           {'POS': 'ADV', 'OP': '*'},
           {'LEMMA': 'be'},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'ADJ'}]
p2 = [{'LOWER':{'IN': ['he', 'she']}},
           {'POS': 'ADV', 'OP': '*'},
           {'LEMMA': 'be'},
           {'POS': 'ADV', 'OP': '*'},
           {'DEP': 'CARDINAL', 'OP': '*'},
           {'POS': 'ADJ'},
           {'POS': 'NOUN', 'OP': '*'}]
p3 = [{'LOWER':{'IN': ['he', 'she']}},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'VERB'},
           {'DEP': 'TRUE', 'OP': '*'},
           {'POS': 'ADJ', 'OP': '*'},
           {'POS': 'NOUN'}]
p4 = [{'ORTH': 'Dr.', 'OP': '*'},
           {'ENT_TYPE': 'PERSON'},
           {'POS': 'ADV', 'OP': '*'},
           {'LEMMA': 'be'},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'ADJ', 'OP': '+'},
           {'ORTH': 'and', 'OP': '*'},
           {}]
p5 = [{'POS': 'NOUN'},
           {'LEMMA': 'be'},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'ADJ'},
           {'ORTH': ',', 'OP': '*'},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'ADJ'},
           {'ORTH': ',', 'OP': '*'},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'ADJ'},
           {'ORTH': ', and', 'OP': '*'},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'ADJ'}]
p6 = [{'LOWER':{'IN': ['he', 'she']}},
           {'LEMMA': 'be'},
           {'POS': 'ADJ'},
           {'ORTH': ',', 'OP': '?'},
           {'POS': 'ADJ', 'OP': '?'},
           {'ORTH': ',', 'OP': '?'}, 
           {'ORTH': 'and', 'OP': '?'},
           {'POS': 'ADJ', 'OP': '?'}]
patterns = [p1, p2, p3, p4, p5, p6]
matcher.add("review", None, *patterns)
span_storage = []
for match_id, start, end in matcher(doc):
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    span_storage.append(span.text)

In [12]:
import numpy as np
sim_table = []
for doc1 in span_storage:
    for doc2 in span_storage:
        sim_table.append(nlp(doc1).similarity(nlp(doc2)))
sim_table = np.array(sim_table).astype(float).reshape(len(span_storage), len(span_storage))       

In [13]:
import pandas as pd
sim_table = pd.DataFrame(data=sim_table, columns=span_storage, index=span_storage)

In [14]:
sim_table

,Richard was very empathic,Richard was very empathic and,Richard was very empathic and understanding,Richard was amazingly kind,Richard was amazingly kind and,Richard was amazingly kind and patient,Dr. McConkie was very thorough,McConkie was very thorough,Dr. McConkie was very thorough with,McConkie was very thorough with,McConkie was exceptional,McConkie was exceptional.,He is compassionate,"He is compassionate,","He is compassionate, professional","He is compassionate, professional and","He is compassionate, professional and knowledgeable",Richard is really good,Richard is really good at
Richard was very empathic,1.000000,0.968968,0.934621,0.862182,0.838039,0.830251,0.768914,0.728817,0.755126,0.711325,0.642493,0.636033,0.687397,0.665980,0.654049,0.651136,0.651739,0.793777,0.762364
Richard was very empathic and,0.968968,1.000000,0.971865,0.870100,0.900403,0.891078,0.793269,0.771259,0.808346,0.783203,0.693878,0.703229,0.727726,0.727142,0.719464,0.750835,0.739513,0.818753,0.795180
Richard was very empathic and understanding,0.934621,0.971865,1.000000,0.829036,0.866894,0.877677,0.794047,0.788057,0.814128,0.802807,0.692426,0.706004,0.755704,0.749318,0.752639,0.780610,0.779111,0.803617,0.782999
Richard was amazingly kind,0.862182,0.870100,0.829036,1.000000,0.973329,0.920910,0.803244,0.781206,0.799558,0.771587,0.732172,0.728514,0.690763,0.692774,0.679183,0.687295,0.671169,0.887896,0.853151
Richard was amazingly kind and,0.838039,0.900403,0.866894,0.973329,1.000000,0.953058,0.809739,0.803016,0.832595,0.820888,0.758224,0.769506,0.718681,0.737549,0.728503,0.768081,0.742994,0.885445,0.859264
Richard was amazingly kind and patient,0.830251,0.891078,0.877677,0.920910,0.953058,1.000000,0.833426,0.815400,0.859639,0.838885,0.750364,0.763133,0.761213,0.767576,0.766221,0.798857,0.783451,0.841051,0.822821
Dr. McConkie was very thorough,0.768914,0.793269,0.794047,0.803244,0.809739,0.833426,1.000000,0.925605,0.974265,0.897972,0.723033,0.725338,0.695770,0.698312,0.711320,0.717513,0.730889,0.753759,0.737013
McConkie was very thorough,0.728817,0.771259,0.788057,0.781206,0.803016,0.815400,0.925605,1.000000,0.918474,0.967514,0.795467,0.783285,0.715707,0.714949,0.738271,0.750114,0.764562,0.731724,0.727157
Dr. McConkie was very thorough with,0.755126,0.808346,0.814128,0.799558,0.832595,0.859639,0.974265,0.918474,1.000000,0.946615,0.748010,0.757316,0.710711,0.718906,0.737736,0.760470,0.765113,0.762071,0.753856
McConkie was very thorough with,0.711325,0.783203,0.802807,0.771587,0.820888,0.838885,0.897972,0.967514,0.946615,1.000000,0.802418,0.800841,0.721027,0.727302,0.755257,0.784914,0.789026,0.736401,0.739575


### From my observation, all extracted reviews that are from a same sentence (for example: 'Richard was amazingly kind', 'Richard was amazingly kind and', and 'Richard was amazingly kind and patient' have at least 90% semantic similarity. Thus, I will group those with at least 90% semantic similarity together and only keep the longest one among them.